<a href="https://colab.research.google.com/github/FarukKaramanVektorel/yz/blob/master/fineTunning_02_03_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from google.colab import files
import pandas as pd

# CSV dosyasını yükleyin
uploaded = files.upload()

# CSV dosyasını pandas ile okuyun
file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name)

# İlk birkaç satırı kontrol edin
print(data.head())

Saving qa_pairs_all_variations.csv to qa_pairs_all_variations.csv
                                        Input (Soru)  \
0  Metinde geçen şu ifade ne anlama geliyor: '(5)...   
1       Bu metinde 'için' kavramı nasıl açıklanıyor?   
2  Bu cümlede ne anlatılıyor: ': 2016/1515 sayılı...   
3            Dökümanın 26. bölümünü özetler misiniz?   
4            Dökümanın 34. bölümünü özetler misiniz?   

                                      Output (Cevap)  
0  (5) İpotekli borç senedi ve irat senedi, taşın...  
1  (2) Tapu sicili, her mahalle veya köy için ayr...  
2   : 2016/1515 sayılı Kararı ile iptal \nedilmiştir  
3  BEŞİNCİ BÖLÜM  \nMülkiyet Hakkı ve Sınırlı Ayn...  
4  Tescilin kontrolü  \nMADDE 36 – (1) Müdür veya...  


In [18]:
from datasets import Dataset

# Pandas DataFrame'i Hugging Face Dataset'e dönüştürme
dataset = Dataset.from_pandas(data)

# Dataset'i kontrol edin
print(dataset)

Dataset({
    features: ['Input (Soru)', 'Output (Cevap)'],
    num_rows: 453
})


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Model ve tokenizer'ı yükleyin
model_name = "t5-small"  # Örnek model (T5)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
def preprocess_function(examples):
    inputs = examples["Input (Soru)"]
    targets = examples["Output (Cevap)"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Cevapları da tokenize edin
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Veri kümesini tokenize edin
tokenized_dataset = dataset.map(preprocess_function, batched=True)

NameError: name 'dataset' is not defined

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Eğitim için kullanılan cihaz: {device}")

# Modeli cihaza taşı
model = model.to(device)


from transformers import TrainingArguments, Trainer

# Eğitim argümanlarını ayarla
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    push_to_hub=True,
    hub_model_id="abuzerkomurcu78/tst2",

    report_to="none"  # W&B'yi devre dışı bırak
)

# Veri ön işleme fonksiyonu
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["Input (Soru)"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

    labels = tokenizer(
        text_target=examples["Output (Cevap)"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Veri kümesini tokenleştir
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Trainer'ı oluştur
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# Modeli eğit
trainer.train()

# Modeli Hugging Face'e yükle
trainer.push_to_hub()


Eğitim için kullanılan cihaz: cuda


NameError: name 'model' is not defined